In [ ]:
import os


os.environ['https_proxy'] = 'http://172.17.0.1:1081'
os.environ['CUDA_VISIBLE_DEVICES'] = '0'


In [2]:
from axolotl.cli import load_cfg


cfg = load_cfg('../examples/mlpspec/llama3-8b-instruct.yaml')
#cfg = load_cfg('../examples/medusa/vicuna_7b-v1.3_stage1.local.yml')
cfg

/root/work/axolotl/env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2024-11-05 19:59:20,425] [INFO] [axolotl.utils.config.models.input.check_eval_packing:978] [PID:1422155] [RANK:0] explicitly setting `eval_sample_packing` to match `sample_packing`
[2024-11-05 19:59:20,428] [INFO] [axolotl.normalize_config:183] [PID:1422155] [RANK:0] GPU memory usage baseline: 0.000GB (+1.184GB misc)


{'medusa_logging': False,
 'medusa_scheduler': 'constant',
 'medusa_only_heads': False,
 'medusa_num_unfreeze_layers': 0,
 'medusa_distillation_regularization': 0,
 'medusa_self_distillation': False,
 'type_of_model': 'LlamaForCausalLM',
 'lisa_layers_attribute': 'model.layers',
 'wandb_project': 'mlpspec_test',
 'gradient_accumulation_steps': 4,
 'micro_batch_size': 2,
 'train_on_inputs': False,
 'group_by_length': False,
 'learning_rate': 0.0005,
 'weight_decay': 0.0,
 'optimizer': <OptimizerNames.ADAMW_BNB: 'adamw_bnb_8bit'>,
 'lr_scheduler': <SchedulerType.COSINE: 'cosine'>,
 'num_epochs': 4,
 'load_in_8bit': False,
 'load_in_4bit': False,
 'lora_dropout': 0.0,
 'qlora_sharded_model_loading': False,
 'loraplus_lr_embedding': 1e-06,
 'output_dir': './outputs/llama3-8b-instruct',
 'base_model': '/models/Meta-Llama-3-8B-Instruct',
 'base_model_config': '/models/Meta-Llama-3-8B-Instruct',
 'tokenizer_type': 'LlamaTokenizer',
 'strict': False,
 'datasets': [{'path': '/models/datasets/Sh

In [3]:
from axolotl.prompt_strategies.sharegpt import register_llama3_template


register_llama3_template(cfg.default_system_message)

In [4]:
#from axolotl.utils.models import load_model, load_tokenizer


#tokenizer = load_tokenizer(cfg)
#tokenizer

In [5]:
from transformers import AutoTokenizer


tokenizer = AutoTokenizer.from_pretrained(cfg.base_model_config, use_fast=False)
tokenizer

PreTrainedTokenizerFast(name_or_path='/models/Meta-Llama-3-8B-Instruct', vocab_size=128000, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|begin_of_text|>', 'eos_token': '<|end_of_text|>', 'unk_token': '<|unk|>', 'sep_token': '<|sep|>', 'cls_token': '<|cls|>', 'mask_token': '<|mask|>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	128000: AddedToken("<|begin_of_text|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	128001: AddedToken("<|end_of_text|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	128002: AddedToken("<|reserved_special_token_0|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	128003: AddedToken("<|reserved_special_token_1|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	128004: AddedToken("<|reserved_special_token_2|>", 

In [6]:
from axolotl.utils.models import load_model


model, peft_config = load_model(cfg, tokenizer, inference=False)
model

[2024-11-05 19:59:22,330] [INFO] [accelerate.utils.modeling.get_max_memory:839] [PID:1422155] Device 5 seems unavailable, Proceeding to check subsequent devices.


Loading checkpoint shards: 100%|██████████| 4/4 [01:16<00:00, 19.06s/it]


[2024-11-05 20:03:10,280] [INFO] [axolotl.load_model:842] [PID:1422155] [RANK:0] GPU memory usage after model load: 1.385GB (+0.008GB cache, +1.564GB misc)
[2024-11-05 20:03:10,456] [INFO] [axolotl.load_model:907] [PID:1422155] [RANK:0] converting modules to torch.bfloat16 for flash attention


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128260, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaFlashAttention2(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      )
    )
    